<h2><font style="font-weight:lighter", color='960018'>Импорт библиотек и данных</font></h2>

In [1]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open('finally_model', 'rb') as f:
    finally_model = pickle.load(f)

In [3]:
df_data = pd.read_csv('Data.csv', sep=',', encoding='utf-8')
df_stores = pd.read_csv('Stores.csv', sep=',', encoding='utf-8')

In [4]:
df = pd.merge(df_data, df_stores)

<h2><font style="font-weight:lighter", color='960018'>Разведочный анализ данных</font></h2>

<b>Повторимо частину коду з файлу Analysis

In [5]:
df.rename(columns = {'Idчека':'Idcheck', 'Дата':'Date', 'Время':'Time', 'Магазин':'Shop',
                      'Город':'City', 'РежимРаботы':'Workingmode'}, inplace = True )
for i in range(df.shape[0]):
    df.Time.iloc[i] = pd.Timestamp(df.Time.iloc[i]).hour

In [6]:
df_help = df.groupby(by=['Date', 'Shop', 'City', 'Workingmode', 'Time']).agg({'Idcheck': 'count'})
df_help = df_help.reset_index()

In [7]:
dict_shop = {}
for i, item in enumerate(df_help.Shop.unique()):
    dict_shop[item] = i
for i in range(df_help.shape[0]):
    df_help.Shop.iloc[i] = dict_shop[df_help.Shop.iloc[i]]
    
dict_mode = {}
for i, item in enumerate(df_help.Workingmode.unique()):
    dict_mode[item] = i
for i in range(df_help.shape[0]):
    df_help.Workingmode.iloc[i] = dict_mode[df_help.Workingmode.iloc[i]]

dict_city = {}
for i, item in enumerate(df_help.City.unique()):
    dict_city[item] = i
for i in range(df_help.shape[0]):
    df_help.City.iloc[i] = dict_city[df_help.City.iloc[i]]

In [8]:
df_help.Date = pd.to_datetime(df_help.Date)


df_help.Shop = df_help.Shop.astype('category')
df_help.City = df_help.City.astype('category')
df_help.Workingmode = df_help.Workingmode.astype('category')
df_help.Time = df_help.Time.astype('category')

In [9]:
X = df_help[['Date', 'Time', 'City', 'Shop', 'Workingmode']]
y = df_help['Idcheck']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

<b>Проверим, правильно ли разделились данные

In [10]:
print(f'{round((len(X_test)/len(X)*100))}%')

25%


<b>Для полной демонстрации програмы, експерементально создадим ситуацию, когда нужно создать план график для X_test (25% от начальных данных)

In [11]:
X_temp = X_test
X_train = X_train.drop(['Date'], axis=1)
X_test = X_test.drop(['Date'], axis=1)
finally_model.fit(X_train, y_train)

y_pred = finally_model.predict(X_test)
print(f'-----------DecisionTreeRegressor-----------\n'
      f'MAPE: {mean_absolute_percentage_error(y_pred.round(0), y_test)}\n'
      f'Train score: {finally_model.score(X_train, y_train)}\n'
      f'Test score: {finally_model.score(X_test, y_test)}\n')

-----------DecisionTreeRegressor-----------
MAPE: 0.41833412970828626
Train score: 0.7618086212310373
Test score: 0.7398153757328401



In [12]:
Ready_df = X_temp

In [13]:
Ready_df['Quantity_of_checks'] = y_pred.round(0)

<b>Ready_df - таблица с уже предсказаными данными

In [14]:
Ready_df.head()

,Date,Time,City,Shop,Workingmode,Quantity_of_checks
381,2016-03-02,10,5,8,1,4.0
2164,2016-01-18,12,2,5,1,4.0
538,2016-04-02,20,5,8,1,19.0
2457,2016-01-20,19,2,5,1,16.0
1780,2016-02-13,19,0,1,0,28.0


<b>Чтобы бизнес партнёру было понятно из чего состоит таблица, вернём обратно названия для стобцов

In [15]:
Ready_df.Shop = Ready_df.Shop.astype('str')
Ready_df.City = Ready_df.City.astype('str')
Ready_df.Workingmode = Ready_df.Workingmode.astype('str')
Ready_df.Time = Ready_df.Time.astype('int')

In [16]:
def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

In [17]:
for i in range(Ready_df.shape[0]):
    Ready_df.Shop.iloc[i] = get_key(dict_shop, int(Ready_df.Shop.iloc[i]))
    
for i in range(Ready_df.shape[0]):
    Ready_df.Workingmode.iloc[i] = get_key(dict_mode, int(Ready_df.Workingmode.iloc[i]))

for i in range(Ready_df.shape[0]):
    Ready_df.City.iloc[i] = get_key(dict_city, int(Ready_df.City.iloc[i]))

<b>Теперь Ready_df приобрёл понятный для нас вид 

In [18]:
Ready_df = Ready_df.sort_values(['Shop', 'Date'])

In [19]:
Ready_df.head()

,Date,Time,City,Shop,Workingmode,Quantity_of_checks
1,2016-01-02,1,Днепропетровск,Магазин 1,Круглосуточный,3.0
17,2016-01-02,20,Днепропетровск,Магазин 1,Круглосуточный,43.0
10,2016-01-02,13,Днепропетровск,Магазин 1,Круглосуточный,16.0
14,2016-01-02,17,Днепропетровск,Магазин 1,Круглосуточный,33.0
4,2016-01-02,6,Днепропетровск,Магазин 1,Круглосуточный,2.0


<b>Заполним эту таблцу сотрудниками. Если магазин круглосуточный то сотрудников 2, если с графиком то 1.

<b>Теперь нужно определить сколько нужно "сотрудников в помощь", если один сотрудник может обслуживать только 20 чеков в час, то используя формулу (Количество чеков // 20) мы сможем определить сколько нужно "сотрудников в помощь"

In [20]:
Ready_df['Quantity_workers'] = 0
Ready_df['Quantity_worker_help'] = 0
for i in range(Ready_df.shape[0]):
    if Ready_df.Workingmode.iloc[i] == 'Круглосуточный':
        Ready_df['Quantity_workers'].iloc[i] = 2
    else:
        Ready_df['Quantity_workers'].iloc[i] = 1

for i in range(Ready_df.shape[0]):
    if Ready_df.Quantity_of_checks.iloc[i] > 20:
        Ready_df['Quantity_worker_help'].iloc[i] = (Ready_df.Quantity_of_checks.iloc[i] // 20)
    else:
        Ready_df['Quantity_worker_help'].iloc[i] = 0
Ready_df['Total'] = Ready_df['Quantity_workers'] + Ready_df['Quantity_worker_help']

In [21]:
Ready_df.head()

,Date,Time,City,Shop,Workingmode,Quantity_of_checks,Quantity_workers,Quantity_worker_help,Total
1,2016-01-02,1,Днепропетровск,Магазин 1,Круглосуточный,3.0,2,0,2
17,2016-01-02,20,Днепропетровск,Магазин 1,Круглосуточный,43.0,2,2,4
10,2016-01-02,13,Днепропетровск,Магазин 1,Круглосуточный,16.0,2,0,2
14,2016-01-02,17,Днепропетровск,Магазин 1,Круглосуточный,33.0,2,1,3
4,2016-01-02,6,Днепропетровск,Магазин 1,Круглосуточный,2.0,2,0,2


In [22]:
Ready_df_help = Ready_df.groupby(by=['Date', 'Shop', 'City', 'Workingmode']).agg({'Total': 'max'}).reset_index().sort_values(['Shop', 'Date'])

In [23]:
Ready_df_help.head()

,Date,Shop,City,Workingmode,Total
0,2016-01-02,Магазин 1,Днепропетровск,Круглосуточный,4
10,2016-01-18,Магазин 1,Днепропетровск,Круглосуточный,3
20,2016-01-19,Магазин 1,Днепропетровск,Круглосуточный,4
30,2016-01-20,Магазин 1,Днепропетровск,Круглосуточный,3
40,2016-01-21,Магазин 1,Днепропетровск,Круглосуточный,4


<b> Осталось определить график роботы для "сотрудников в помощь"

In [24]:
df_work = Ready_df[Ready_df.Quantity_of_checks > 20].groupby(['Date', 'Shop', 'City', 'Workingmode']).agg({'Time': 'min'})
df_work['Max'] = Ready_df[Ready_df.Quantity_of_checks > 20].groupby(['Date', 'Shop', 'City', 'Workingmode']).agg({'Time': 'max'})

In [25]:
df_work = df_work.rename(columns = {'Time':'Min'}).reset_index().sort_values(['Shop', 'Date'])

In [26]:
df_work.Max += 1
df_work['Diff'] = df_work.Max - df_work.Min

<b>Получено начало и конец времени когда нужны "сотрудники в помощь"

In [27]:
df_work.shape

(103, 7)

<b>Но по условию сотрудника «в помощь» мы можем выводить на временные промежутки кратностью в 4 часа (4 -8-12).

In [28]:
for i in range(df_work.shape[0]):
    if df_work.Diff.iloc[i] <= 4:
        df_work.Diff.iloc[i] = 4
    elif (df_work.Diff.iloc[i] > 4) and ((df_work.Diff.iloc[i] <= 8)):
        df_work.Diff.iloc[i] = 8
    elif (df_work.Diff.iloc[i] > 8) and ((df_work.Diff.iloc[i] <= 12)):
        df_work.Diff.iloc[i] = 12

In [29]:
df_work['DiffNX'] = df_work.Max - df_work.Min

<b>Столбец DiffNX показывает нам сколько сотрудник проработает, если сотрудник может работать только 4-8-12 часов, мы должны в таком случае вызвать его раньше или позже, чтобы избечь ситуации, когда сотрудник проработает 1 час, а получит финансовое вознаграждение за 4 часа

In [30]:
 df_work['Start_of_Day_for_Helpers'] = 0
for i in range(df_work.shape[0]):
    if df_work.Diff.iloc[i] != df_work.DiffNX.iloc[i]:
        if df_work.Workingmode.iloc[i] == 'Круглосуточный':
            if (df_work.Diff.iloc[i] - df_work.DiffNX.iloc[i]) > 0:
                df_work['Start_of_Day_for_Helpers'].iloc[i] = df_work.Min.iloc[i] - (df_work.Diff.iloc[i] - df_work.DiffNX.iloc[i])
            else:
                continue
        else:
        
            if ((df_work.Diff.iloc[i] - df_work.DiffNX.iloc[i]) > 0) and \
                (df_work.Min.iloc[i] - (df_work.Diff.iloc[i] - df_work.DiffNX.iloc[i]) < 10):
                df_work['Start_of_Day_for_Helpers'].iloc[i] = df_work.Min.iloc[i] + (df_work.Diff.iloc[i] - df_work.DiffNX.iloc[i])
            else:
                df_work['Start_of_Day_for_Helpers'].iloc[i] = df_work.Min.iloc[i] - (df_work.Diff.iloc[i] - df_work.DiffNX.iloc[i])
    else:
        df_work['Start_of_Day_for_Helpers'].iloc[i] = df_work.Min.iloc[i]
df_work = df_work.rename(columns={'Max': 'End_of_Day_for_Helpers'})

In [31]:
Ready_df_help

,Date,Shop,City,Workingmode,Total
0,2016-01-02,Магазин 1,Днепропетровск,Круглосуточный,4
10,2016-01-18,Магазин 1,Днепропетровск,Круглосуточный,3
20,2016-01-19,Магазин 1,Днепропетровск,Круглосуточный,4
30,2016-01-20,Магазин 1,Днепропетровск,Круглосуточный,3
40,2016-01-21,Магазин 1,Днепропетровск,Круглосуточный,4
...,...,...,...,...,...
216,2016-06-02,Магазин 9,Днепропетровск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,1
244,2016-09-02,Магазин 9,Днепропетровск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,1
253,2016-10-02,Магазин 9,Днепропетровск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,1
263,2016-11-02,Магазин 9,Днепропетровск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,1


In [32]:
df_work

,Date,Shop,City,Workingmode,Min,End_of_Day_for_Helpers,Diff,DiffNX,Start_of_Day_for_Helpers
0,2016-01-02,Магазин 1,Днепропетровск,Круглосуточный,17,21,4,4,17
3,2016-01-18,Магазин 1,Днепропетровск,Круглосуточный,22,23,4,1,19
5,2016-01-19,Магазин 1,Днепропетровск,Круглосуточный,16,19,4,3,15
7,2016-01-20,Магазин 1,Днепропетровск,Круглосуточный,15,17,4,2,13
12,2016-01-21,Магазин 1,Днепропетровск,Круглосуточный,18,23,8,5,15
...,...,...,...,...,...,...,...,...,...
73,2016-03-02,Магазин 8,Новомосковск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,17,18,4,1,14
77,2016-04-02,Магазин 8,Новомосковск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,18,19,4,1,15
80,2016-05-02,Магазин 8,Новомосковск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,17,19,4,2,15
93,2016-09-02,Магазин 8,Новомосковск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,17,18,4,1,14


In [33]:
a = df_work[['Date', 'Shop', 'City', 'Workingmode', 'Start_of_Day_for_Helpers', 'End_of_Day_for_Helpers']]

In [34]:
b = Ready_df_help[['Date', 'Shop', 'City', 'Workingmode', 'Total']]

In [35]:
a

,Date,Shop,City,Workingmode,Start_of_Day_for_Helpers,End_of_Day_for_Helpers
0,2016-01-02,Магазин 1,Днепропетровск,Круглосуточный,17,21
3,2016-01-18,Магазин 1,Днепропетровск,Круглосуточный,19,23
5,2016-01-19,Магазин 1,Днепропетровск,Круглосуточный,15,19
7,2016-01-20,Магазин 1,Днепропетровск,Круглосуточный,13,17
12,2016-01-21,Магазин 1,Днепропетровск,Круглосуточный,15,23
...,...,...,...,...,...,...
73,2016-03-02,Магазин 8,Новомосковск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,14,18
77,2016-04-02,Магазин 8,Новомосковск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,15,19
80,2016-05-02,Магазин 8,Новомосковск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,15,19
93,2016-09-02,Магазин 8,Новомосковск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,14,18


In [36]:
b

,Date,Shop,City,Workingmode,Total
0,2016-01-02,Магазин 1,Днепропетровск,Круглосуточный,4
10,2016-01-18,Магазин 1,Днепропетровск,Круглосуточный,3
20,2016-01-19,Магазин 1,Днепропетровск,Круглосуточный,4
30,2016-01-20,Магазин 1,Днепропетровск,Круглосуточный,3
40,2016-01-21,Магазин 1,Днепропетровск,Круглосуточный,4
...,...,...,...,...,...
216,2016-06-02,Магазин 9,Днепропетровск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,1
244,2016-09-02,Магазин 9,Днепропетровск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,1
253,2016-10-02,Магазин 9,Днепропетровск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,1
263,2016-11-02,Магазин 9,Днепропетровск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,1


In [37]:
Plan_Schedule = pd.merge(b, a, how='left')

In [38]:
Plan_Schedule = Plan_Schedule.fillna('No helpers')

<h2><font style="font-weight:lighter", color='960018'>План-График для сотрудников и "сотрудников в помощь"</font></h2>

<b> На основе предсказаний модели, сделан план-график, где максимальное количество сотрудников указываеться в колонке Total, а график роботы для "сотрудников в помощь" в колонках Start_of_Day_for_Helpers и End_of_Day_for_Helpers

In [39]:
Plan_Schedule

,Date,Shop,City,Workingmode,Total,Start_of_Day_for_Helpers,End_of_Day_for_Helpers
0,2016-01-02,Магазин 1,Днепропетровск,Круглосуточный,4,17.0,21.0
1,2016-01-18,Магазин 1,Днепропетровск,Круглосуточный,3,19.0,23.0
2,2016-01-19,Магазин 1,Днепропетровск,Круглосуточный,4,15.0,19.0
3,2016-01-20,Магазин 1,Днепропетровск,Круглосуточный,3,13.0,17.0
4,2016-01-21,Магазин 1,Днепропетровск,Круглосуточный,4,15.0,23.0
...,...,...,...,...,...,...,...
269,2016-06-02,Магазин 9,Днепропетровск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,1,No helpers,No helpers
270,2016-09-02,Магазин 9,Днепропетровск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,1,No helpers,No helpers
271,2016-10-02,Магазин 9,Днепропетровск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,1,No helpers,No helpers
272,2016-11-02,Магазин 9,Днепропетровск,Пн-Чт 10-22; Пт 10-23; Сб 09-23; Вс 09-22,1,No helpers,No helpers


<b>При желании, график можна сохранить в .csv формате для Excel